# ML Trader

In [1]:
from utils.packages import *
from utils.variables import *
from utils.i_o import *
from utils.dir_slope import *
from utils.engine import *
from utils.loops import *
from utils.order import *
import winsound

In [2]:
%%time
data = run_engine(data)

100%|████████████████████████████▉| 1922239/1924042 [02:49<00:00, 11434.68it/s]

pl   -0.03872
dtype: float64
--------------------------------------
              DateTime_frmt          close_type       pl
1895849 2021-01-29 15:57:20         simple_stop -0.00089
1897688 2021-01-29 16:03:01         simple_stop -0.00062
1901754 2021-01-29 16:33:29         simple_stop -0.00061
1903610 2021-01-29 16:48:31         simple_stop -0.00072
1904456 2021-01-29 16:57:22  simple_take_profit  0.00036
1905457 2021-01-29 17:08:48         simple_stop -0.00051
1907591 2021-01-29 17:33:35         simple_stop -0.00046
1910179 2021-01-29 18:00:08  simple_take_profit  0.00023
1910323 2021-01-29 18:01:21  simple_take_profit  0.00023
1912352 2021-01-29 18:27:35  simple_take_profit  0.00026
1914490 2021-01-29 19:01:22  simple_take_profit  0.00019
1915759 2021-01-29 19:21:27  simple_take_profit  0.00019
1919457 2021-01-29 20:25:32         simple_stop -0.00045
1920654 2021-01-29 20:50:20  simple_take_profit  0.00019
1922238 2021-01-29 21:09:23  simple_take_profit  0.00027


100%|█████████████████████████████| 1924042/1924042 [02:49<00:00, 11330.17it/s]


Wall time: 3min 18s


In [3]:
generate_result_report(data)

--------------------------------------
Run Complete : data/ema_df-(2021-2021)-(1-1)-(1-31).csv
--------------------------------------


--------------------------------------
Net PL
-0.0387
--------------------------------------


--------------------------------------
Monthly PL
                pl
month_val         
1         -0.03872
--------------------------------------


--------------------------------------
Daily PL
               pl
date_val         
3        -0.00015
4         0.00085
5        -0.00230
6        -0.00603
7        -0.00143
8        -0.00814
10       -0.00176
11        0.00265
12       -0.00137
13       -0.00184
14       -0.00108
15        0.00066
17        0.00008
18        0.00373
19        0.00048
20       -0.00390
21       -0.00298
22       -0.00771
24       -0.00123
25        0.00128
26        0.00009
27       -0.00484
28        0.00095
29       -0.00473
--------------------------------------


--------------------------------------
Count of transactions by 

In [6]:
data["plot"] = True
data['plot_type'] = 'file'
data['reduce_plot'] = True
data['plot_start']  = '2021-01-08'
data['plot_stop']   = '2021-01-09'
data['things_to_plot'] = ['tick', 'lema', 'h_l_lema', 'tick_angle', 'positions']

In [7]:
plot_graph(data)

In [17]:
data['df'] = pd.read_csv('data/ema_df-(2021-2021)-(1-1)-(1-31).csv')

data['angle_len'] = 500
data = get_x_axis(data)
data['df']['tick_angle'] = data['df']['tick'].rolling(window=data['angle_len']).progress_apply(roll_slope)
data['df'] = data['df'].dropna()

data['df'] = data['df'][data['columns_list']]

data['df'].to_csv('data/ema_df-(2021-2021)-(1-1)-(1-31).csv')
data['df'].head(2)